In [52]:
from lxml import html
import requests
from bs4 import BeautifulSoup
import json

import pandas as pd

DownloadNewData = False

if DownloadNewData:
    session = requests.Session()
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.19577'}
    ads_returned = True
    pagenr = 0
    df_ads = pd.DataFrame()
    while ads_returned:
        pagenr = pagenr+1
        url = f'https://www.marktplaats.nl/l/caravans-en-kamperen/vouwwagens/p/{pagenr}/#q:holtkamper|PriceCentsTo:1000000|searchInTitleAndDescription:true'
        page = session.get(url, headers=headers)
        soup = BeautifulSoup(page.content, 'html.parser')
        # print(soup.prettify())
        data = json.loads(soup.find('script',type='application/json').text)
        ads = data['props']['pageProps']['searchRequestAndResponse']['listings']
        if len(ads) > 0:
            df_temp = pd.DataFrame(ads)
            df_ads = pd.concat([df_ads,df_temp],ignore_index=True)
        else:
            ads_returned = False
    df_ads.to_pickle('Download vouwwagens 20230821.pkl')
else:
    df_ads = pd.read_pickle('Download vouwwagens 20230821.pkl')

# df_ads

In [56]:
### drop adevrtisement with "a" (commercial?)
df_ads.drop(df_ads[df_ads['itemId'].str.contains('a')].index,inplace=True)
# df_ads

,itemId,title,description,categorySpecificDescription,thinContent,priceInfo,location,date,imageUrls,sellerInformation,...,extendedAttributes,traits,verticals,pictures,vipUrl,attributes,trackingData,pageLocation,trustIndicators,highlights
0,m2013406244,Jamet Dakota outdoor 2019 met easy pack system,"Te koop jamet dakota outdoor 2019, geleverd do...","Te koop jamet dakota outdoor 2019, geleverd do...",False,"{'priceCents': 725000, 'priceType': 'FIXED'}","{'cityName': 'Someren', 'countryName': 'Nederl...",2023-08-21T11:03:21.497Z,[//images.marktplaats.com/api/v1/listing-mp-p/...,"{'sellerId': 20046633, 'sellerName': 'FrankAno...",...,[],"[DAG_TOPPER_7DAYS, PACKAGE_PREMIUM]","[caravans_and_camping_folders, automotive, car...","[{'id': 9005392135, 'mediaId': '', 'url': 'htt...",/v/caravans-en-kamperen/vouwwagens/m2013406244...,NaN,NaN,NaN,NaN,NaN
1,m2013363332,Vouwwagen Jamet Dakota All Season (zelfreinigend),Degelijke en zeer ruime vouwwagen tot 8 person...,Degelijke en zeer ruime vouwwagen tot 8 person...,False,"{'priceCents': 600000, 'priceType': 'MIN_BID'}","{'cityName': ''s-Hertogenbosch', 'countryName'...",2023-08-21T11:03:21.497Z,[//images.marktplaats.com/api/v1/listing-mp-p/...,"{'sellerId': 1591767, 'sellerName': 'Martijn',...",...,"[{'key': 'numberOfSleepingPlaces', 'value': 'M...","[DAG_TOPPER_7DAYS, PACKAGE_PREMIUM]","[caravans_and_camping_folders, automotive, car...","[{'id': 9005067293, 'mediaId': '', 'url': 'htt...",/v/caravans-en-kamperen/vouwwagens/m2013363332...,"[{'key': 'constructionYear', 'value': '2017', ...",NaN,NaN,NaN,NaN
2,m2013253836,Holtkamper Cocoon S (2013),Tien jaar hebben we genoten van deze zeer gebr...,Tien jaar hebben we genoten van deze zeer gebr...,False,"{'priceCents': 700000, 'priceType': 'MIN_BID'}","{'cityName': 'Veenendaal', 'countryName': 'Ned...",2023-08-21T11:03:21.497Z,[//images.marktplaats.com/api/v1/listing-mp-p/...,"{'sellerId': 15349691, 'sellerName': 'Teeuwiss...",...,"[{'key': 'numberOfSleepingPlaces', 'value': 'T...","[DAG_TOPPER_7DAYS, PACKAGE_BASIC]","[caravans_and_camping_folders, automotive, car...","[{'id': 9004219160, 'mediaId': '', 'url': 'htt...",/v/caravans-en-kamperen/vouwwagens/m2013253836...,"[{'key': 'constructionYear', 'value': '2013', ...",NaN,NaN,NaN,NaN
3,m2013201620,Combi Camp Flexi Comfort XL - 2020 - Katoen - ...,"Vandaag te bezichtigen: zéér complete, prachti...","Vandaag te bezichtigen: zéér complete, prachti...",False,"{'priceCents': 950000, 'priceType': 'MIN_BID'}","{'cityName': 'Rosmalen', 'countryName': 'Neder...",2023-08-21T11:03:21.497Z,[//images.marktplaats.com/api/v1/listing-mp-p/...,"{'sellerId': 5412332, 'sellerName': 'Desiree',...",...,"[{'key': 'numberOfSleepingPlaces', 'value': 'T...","[DAG_TOPPER_7DAYS, PACKAGE_PREMIUM]","[caravans_and_camping_folders, automotive, car...","[{'id': 9004943719, 'mediaId': '', 'url': 'htt...",/v/caravans-en-kamperen/vouwwagens/m2013201620...,"[{'key': 'constructionYear', 'value': '2020', ...",NaN,NaN,NaN,NaN
5,m2008112387,Jamet Montana vouwwagen,Kenmerken: - 8 persoons - 100% waterdicht - on...,Kenmerken: - 8 persoons - 100% waterdicht - on...,False,"{'priceCents': 65000, 'priceType': 'MIN_BID'}","{'cityName': ''s-Gravenhage', 'countryName': '...",2023-08-21T11:03:21.497Z,[//images.marktplaats.com/api/v1/listing-mp-p/...,"{'sellerId': 49268641, 'sellerName': 'Brechtje...",...,"[{'key': 'numberOfSleepingPlaces', 'value': 'M...","[DAG_TOPPER_7DAYS, PACKAGE_PREMIUM]","[caravans_and_camping_folders, automotive, car...","[{'id': 8972178974, 'mediaId': '', 'url': 'htt...",/v/caravans-en-kamperen/vouwwagens/m2008112387...,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1015,m2002524896,Alpenkreuzer Waterfront,Waterfront het element water symboliseert leve...,Waterfront het element water symboliseert leve...,False,"{'priceCents': 859500, 'priceType': 'FIXED'}","{'cityName': 'Zwolle', 'countryName': 'Nederla...",2023-07-23T08:40:06Z,[//images.marktplaats.com/api/v

In [65]:
def get_production_year(x):
    if pd.isna(x):
        return pd.NA
    else:
        return int(x[0]['value'])
    
def get_price(x):
    # return 1
    # print(x)
    if pd.isna(x):
        return pd.NA
    else:
        return x['priceCents']/100
    
def get_manufacturer(x):
    ManufacturerList = ['Jamet', 'Holtkamper', 'Combi Camp','Aart Kok','Alpenkreuzer','Kip','Cabanon','Campooz','Trigano','Raclet','Camp-Let','Paradiso','Roadmaster','Gerjak','Rapido','3Dog','Conway']
    for Manufacturer in ManufacturerList:
        if x.lower().find(Manufacturer.lower()) > -1:
            return Manufacturer
    return 'Not found in list'


df_ads['price_eur'] = df_ads['priceInfo'].apply(get_price)
df_ads['construction_year'] = df_ads['attributes'].apply(get_production_year)
df_ads['Manufacturer'] = df_ads['title'].apply(get_manufacturer)

df_ads[df_ads['Manufacturer']=='Not found in list'][['itemId','title','description','Manufacturer','construction_year','price_eur']]


,itemId,title,description,Manufacturer,construction_year,price_eur
7,m2007138116,Vouwwagen 2016,Vouwwagen teun ttt4. Tentdoek gemaakt door dor...,Not found in list,2016,4750.0
15,m2009966572,vouwwagen voor de snelle beslisser en vakantie...,Vouwwagen trigano odyssee. Vakantieklaar! 1-2-...,Not found in list,2006,1900.0
17,m2011714862,"VOUWWAGENS komen uit verhuur 1,5 jaar oud 6-7...","Vouwwagens komen uit verhuur 1,5 jaar oud 6-8 ...",Not found in list,2022,5825.0
23,m2011609794,CAMPLET Concorde,Deze fijne snel op te zetten (1 2 3 systeem) v...,Not found in list,1997,650.0
27,m2012120312,Vouwwagen,Zie er heel mooi uit en kan nog jaren mee door...,Not found in list,<NA>,0.0
...,...,...,...,...,...,...
982,m2003891505,Retro Vouwwagen,Super leuke retro alpenkreutzer vouwwagen incl...,Not found in list,<NA>,0.0
983,m1930976570,Scout vouwwagen 6 persoon inruil auto/brommer ...,Hierbij bied ik te koop leuke nette vouwwagen ...,Not found in list,1996,850.0
985,m2003700586,Coleman Fleetwood Pop-up Vouwwagen Uniek!,Ontdek de ultieme kampeerervaring met onze pra...,Not found in list,<NA>,8500.0
997,m2003489320,Tag Mountain Ranger 2000,Nette taga mountain ranger 2000. Kan zeker nog...,Not found in list,<NA>,699.0


In [67]:
import plotly.express as px
fig = px.scatter(df_ads, x="construction_year", y="price_eur",
                color="Manufacturer",
                #  size='petal_length', 
                hover_data=['title']
                 )
fig.show()